# Comparing WL mass centroids to X-ray centroids

## Import Statements

In [15]:
import numpy as np
import pandas as pd
from astropy.units import Quantity
from matplotlib import pyplot as plt

import xga
xga.NUM_CORES = 5
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.sourcetools import ang_to_rad

%matplotlib inline

FileNotFoundError: root_xmm_dir=/Users/dt237/apollo_mnt/xmm_obs/data/ does not appear to exist, if it an SFTP mount check the connection.

## Setting up necessary directories

## Reading in the results

We read in the LoVoCCS results we need to be able to compare the overall mass distribution and the X-ray ICM distribution:

In [9]:
xray_res = pd.read_csv("../../outputs/coordinates/xmm_500kpc_centroid.csv")
xray_res['LoVoCCSID'] = xray_res['LoVoCCS_name'].apply(lambda x: x.split('-')[1])
xray_res

,LoVoCCS_name,cent_ra,cent_ra-,cent_ra+,cent_dec,cent_dec-,cent_dec+,position_angle,position_angle-,position_angle+,ax_ratio,ax_ratio-,ax_ratio+,LoVoCCSID
0,LoVoCCS-1,227.73319,0.00002,0.00002,5.74323,0.00002,0.00002,-163.2287,0.1878,0.2336,0.9179,0.0010,0.0012,1
1,LoVoCCS-2,44.73666,0.00015,0.00020,13.57764,0.00016,0.00016,-66.6192,4.4151,4.5963,0.9725,0.0055,0.0046,2
2,LoVoCCS-4,10.46009,0.00003,0.00003,-9.30464,0.00005,0.00003,157.3862,0.1785,0.1713,0.8917,0.0011,0.0009,4
3,LoVoCCS-5,303.14178,0.00013,0.00013,-56.83836,0.00007,0.00007,-49.9254,0.2451,0.1678,0.8331,0.0013,0.0011,5
4,LoVoCCS-7,330.47797,0.00019,0.00020,-59.94793,0.00011,0.00012,-168.0726,0.8656,0.8021,0.9492,0.0036,0.0038,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,LoVoCCS-119,125.25839,0.00007,0.00006,7.86452,0.00008,0.00008,-80.7086,1.8766,1.8226,0.9703,0.0052,0.0064,119
57,LoVoCCS-121,52.48007,0.00035,0.00034,-52.58021,0.00017,0.00023,-140.9563,4.2228,4.3894,0.9607,0.0053,0.0069,121
58,LoVoCCS-123,193.67578,0.00031,0.00039,-29.22863,0.00027,0.00029,147.6039,2.2260,2.2397,0.9173,0.0059,0.0064,123
59,LoVoCCS-131,137.65149,0.00050,0.00041,-10.58244,0.00044,0.00050,261.3377,3.1668,2.9612,0.9590,0.0136,0.0141,131


Then we read in a file containing the centroids and position angles of the weak-lensing 2D mass distributions measured by [Fu et al. 2024](https://ui.adsabs.harvard.edu/abs/2024arXiv240210337F/abstract) (the second LoVoCCS publication) - this file also includes the centroids and angles of the red sequence galaxy population. Several measures of the orientation angle are present in the LoVoCCSII paper, but we make use of the mass and red-sequence orientations measured within 1000 kpc. 

We note that some entries are null because of low signal to noise in the weak-lensing shear maps, and some are null because we have split the particular LoVoCCS cluster into several components in our sample:

In [4]:
wlmass_res = pd.read_csv("../../sample_files/X-LoVoCCSI_centroid_pa_results.csv")
wlmass_res

,LoVoCCSID,Name,mass_cent_ra,mass_cent_dec,mass_pos_angle_1Mpc,redseq_cent_ra,redseq_cent_dec,redseq_pos_angle_1Mpc
0,1,A2029,227.73,5.78,84.0,227.74,5.77,-74.0
1,2,A401,44.74,13.58,1.0,44.74,13.58,-48.0
2,4A,A85North,10.45,-9.33,-84.0,10.41,-9.31,72.0
3,4B,A85South,NaN,NaN,NaN,NaN,NaN,NaN
4,5,A3667,303.14,-56.85,80.0,303.15,-56.84,44.0
...,...,...,...,...,...,...,...,...
62,121,A3128,52.64,-52.53,78.0,52.62,-52.56,-87.0
63,122,A1023,157.00,-6.79,-20.0,156.99,-6.79,53.0
64,123,A3528,193.61,-29.00,-76.0,193.59,-29.00,77.0
65,131,A761,137.65,-10.59,32.0,137.68,-10.59,18.0


We also have to read in the sample file, because neither of these results files contain redshift information:

In [5]:
samp = pd.read_csv("../../sample_files/X-LoVoCCSI.csv")
samp

,LoVoCCSID,Name,start_ra,start_dec,MCXC_Redshift,MCXC_R500,MCXC_RA,MCXC_DEC,manual_xray_ra,manual_xray_dec,MCXC_Lx500_0.1_2.4
0,1,A2029,227.734300,5.745471,0.0766,1.3344,227.73000,5.720000,227.734300,5.745471,8.726709e+44
1,2,A401,44.740000,13.580000,0.0739,1.2421,44.74000,13.580000,NaN,NaN,6.088643e+44
2,4A,A85North,10.458750,-9.301944,0.0555,1.2103,10.45875,-9.301944,NaN,NaN,5.100085e+44
3,4B,A85South,10.451487,-9.460007,0.0555,1.2103,10.45875,-9.301944,10.451487,-9.460007,5.100085e+44
4,5,A3667,303.157313,-56.845978,0.0556,1.1990,303.13000,-56.830000,303.157313,-56.845978,4.871933e+44
...,...,...,...,...,...,...,...,...,...,...,...
62,121,A3128,52.466189,-52.580728,0.0624,0.8831,52.50000,-52.600000,52.466189,-52.580728,1.101682e+44
63,122,A1023,157.000000,-6.800000,0.1176,0.8553,157.00000,-6.800000,NaN,NaN,1.095941e+44
64,123,A3528,193.670000,-29.220000,0.0544,0.8855,193.67000,-29.220000,NaN,NaN,1.093054e+44
65,131,A761,137.651250,-10.581111,0.0916,0.8627,137.65125,-10.581111,NaN,NaN,1.063423e+44


In [13]:
comb = pd.merge(samp[['LoVoCCSID', 'MCXC_Redshift']], xray_res, on='LoVoCCSID')
print(comb.shape)
comb = pd.merge(comb, wlmass_res, on='LoVoCCSID')
comb.head(5)

(60, 15)


,LoVoCCSID,MCXC_Redshift,LoVoCCS_name,cent_ra,cent_ra-,cent_ra+,cent_dec,cent_dec-,cent_dec+,position_angle,...,ax_ratio,ax_ratio-,ax_ratio+,Name,mass_cent_ra,mass_cent_dec,mass_pos_angle_1Mpc,redseq_cent_ra,redseq_cent_dec,redseq_pos_angle_1Mpc
0,1,0.0766,LoVoCCS-1,227.73319,0.00002,0.00002,5.74323,0.00002,0.00002,-163.2287,...,0.9179,0.0010,0.0012,A2029,227.73,5.78,84.0,227.74,5.77,-74.0
1,2,0.0739,LoVoCCS-2,44.73666,0.00015,0.00020,13.57764,0.00016,0.00016,-66.6192,...,0.9725,0.0055,0.0046,A401,44.74,13.58,1.0,44.74,13.58,-48.0
2,5,0.0556,LoVoCCS-5,303.14178,0.00013,0.00013,-56.83836,0.00007,0.00007,-49.9254,...,0.8331,0.0013,0.0011,A3667,303.14,-56.85,80.0,303.15,-56.84,44.0
3,7,0.0980,LoVoCCS-7,330.47797,0.00019,0.00020,-59.94793,0.00011,0.00012,-168.0726,...,0.9492,0.0036,0.0038,A3827,330.46,-59.94,69.0,330.48,-59.94,64.0
4,9,0.0589,LoVoCCS-9,67.84353,0.00024,0.00018,-61.43197,0.00012,0.00010,237.5834,...,0.9162,0.0019,0.0016,A3266,67.80,-61.43,80.0,67.82,-61.45,-24.0


## Comparing ICM and weak-lensing mass distribution centroids

### Calculate the offset

We calculate the Haversine distance between the ICM and WL mass distribution centroids - this angular offset is then used to calculate the proper distance offset in kpc. The cosmology used here is the XGA default, and the same that we use throughout the LoVoCCS analyses:

In [ ]:
icm_ra = comb['cent_ra'].values
icm_dec = comb['cent_dec'].values

wl_ra = comb['mass_cent_ra'].values
wl_dec = comb['mass_cent_dec'].values

hav_sep = 2 * np.arcsin(np.sqrt((np.sin(((icm_dec*(np.pi / 180))-(wl_dec*(np.pi / 180))) / 2) ** 2)
                                    + np.cos((wl_dec * (np.pi / 180))) * np.cos(icm_dec * (np.pi / 180))
                                    * np.sin(((icm_ra*(np.pi / 180)) - (wl_ra*(np.pi / 180))) / 2) ** 2))
# Converting back to degrees from radians
hav_sep /= (np.pi / 180)
hav_sep = Quantity(hav_sep, 'deg').to('arcmin')

# Also calculate the proper distance at each cluster's redshift in kpc
prop_hav_sep = ang_to_rad(hav_sep, comb['MCXC_Redshift'].values)